### Code LLama basic usage example with AutoGPTQ and Hugging Face Transformers

just a simple example. for more info about how all this stuff works you may look into this article: https://huggingface.co/blog/gptq-integration

(tested on Python 3.11.1)

##### 1. install all dependencies

In [1]:
!python -m pip install --upgrade pip
!pip install --upgrade setuptools
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

  Using cached pip-23.2.1-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 22.3.1
    Uninstalling pip-22.3.1:
      Successfully uninstalled pip-22.3.1
  Obtaining dependency information for setuptools from https://files.pythonhosted.org/packages/4f/ab/0bcfebdfc3bfa8554b2b2c97a555569c4c1ebc74ea288741ea8326c51906/setuptools-68.1.2-py3-none-any.whl.metadata
  Using cached setuptools-68.1.2-py3-none-any.whl.metadata (6.2 kB)
Using cached setuptools-68.1.2-py3-none-any.whl (805 kB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 65.5.0
    Uninstalling setuptools-65.5.0:
      Successfully uninstalled setuptools-65.5.0
Looking in indexes: https://download.pytorch.org/whl/cu117
  Using cached https://download.pytorch.org/whl/cu117/torch-2.0.1%2Bcu117-cp311-cp311-win_amd64.whl (2343.6 MB)
  Using cached https://download.pytorch.org/whl/cu117/torchvision-0.15.2%2Bcu117-cp311-cp311-win_amd64.whl (4.9 MB)
  Using cached h

In [2]:
!pip install -q -U transformers peft accelerate optimum

In [3]:
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu117/

Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu117/
     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
      --------------------------------------- 0.0/1.5 MB ? eta -:--:--
     - -------------------------------------- 0.0/1.5 MB 653.6 kB/s eta 0:00:03
     - -------------------------------------- 0.0/1.5 MB 653.6 kB/s eta 0:00:03
     - -------------------------------------- 0.1/1.5 MB 465.5 kB/s eta 0:00:04
     - -------------------------------------- 0.1/1.5 MB 465.5 kB/s eta 0:00:04
     -- ------------------------------------- 0.1/1.5 MB 305.0 kB/s eta 0:00:05
     -- ------------------------------------- 0.1/1.5 MB 308.0 kB/s eta 0:00:05
     -- ------------------------------------- 0.1/1.5 MB 308.0 kB/s eta 0:00:05
     -- ------------------------------------- 0.1/1.5 MB 308.0 kB/s eta 0:00:05
     -- ------------------------------------- 

In [ ]:
#from transformers import pipeline

#pipe = pipeline("text-generation", model="TheBloke/CodeLlama-7B-Python-GGML") # CodeLlama-7B-Python

In [ ]:
# !netsh int tcp set global autotuninglevel=normal # solution for windows users from https://www.reddit.com/r/StableDiffusion/comments/z8dxhn/sd_20_automatic1111_download/ 

In [2]:
gc.collect()

torch.cuda.empty_cache()

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import gc

gc.collect()

torch.cuda.empty_cache()

model_checkpoint = "TheBloke/Llama-2-7b-Chat-GPTQ"

model = AutoModelForCausalLM.from_pretrained(model_checkpoint, torch_dtype=torch.float16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

q:\SANDBOX\code_llama\my_env_2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 4.00 GiB total capacity; 3.26 GiB already allocated; 0 bytes free; 3.48 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
text = "generate code for PageRank alghoritm implementation"
inputs = tokenizer(text, return_tensors="pt").to(0)

out = model.generate(**inputs)
print(tokenizer.decode(out[0], skip_special_tokens=True))

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig
import torch

model_id = "facebook/opt-125m"

quantization_config = GPTQConfig(
     bits=4,
     group_size=128,
     dataset="c4",
     desc_act=False,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
quant_model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config, device_map='auto')

# Fine-tune quantized models with PEFT